## Some example implementations of Newton's Method for Logisitic Regression

<a href="#params_setup">Parameter Setup</a><br>
<a href="#algorithms">Algorithms</a><br>
<a href="#newton">Newton's Method</a><br>
<a href="#conv">Convergence Criteria</a><br>
<a href="#numerics">Numerical Examples</a><br>

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
from patsy import dmatrices
import warnings

# import statsmodels.formula.api as smf # if you want to check results

<a name="data_setup"></a>
## Setup

### Parameter / Data Setup

In the below cells, there are various parameters and options to play with involving data creation, algorithm settings, and what model you want to try and fit.

In [2]:
def sigmoid(x):
    '''Sigmoid function of x.'''
    return 1/(1+np.exp(-x))

def log_likelihood(p, y):
    '''Computes the log-likelihood'''
    
    return np.sum(y*np.log(p) + (1-y)*np.log(1-p)).values[0]

In [3]:
np.random.seed(20009) # set the seed

## algorithm settings
tol=1e-8 # convergence tolerance
lam = None # l2-regularization
max_iter = 20 # maximum allowed iterations

## data creation settings
r = 0.95 # covariance between x and z
n = 1000 # number of observations
sigma = 1 # variance of noise

## model settings
beta_x, beta_z, beta_v = -4, .9, 1 # true beta coefficients
var_x, var_z, var_v = 1, 1, 4 # variances of inputs

## the model specification you want to fit
formula = 'y ~ x + z + v + np.exp(x) + I(v**2 + z)'

In [4]:
## create the data

x, z = np.random.multivariate_normal([0,0], [[var_x,r],[r,var_z]], n).T
v = np.random.normal(0,var_v,n)**3

A = pd.DataFrame({'x' : x, 'z' : z, 'v' : v})
A['log_odds'] = sigmoid(A[['x','z','v']].dot([beta_x,beta_z,beta_v]) + sigma*np.random.normal(0,1,n))
A['y'] = [np.random.binomial(1,p) for p in A.log_odds]

y, X = dmatrices(formula, A, return_type='dataframe')

X.head()

,Intercept,x,z,v,np.exp(x),I(v ** 2 + z)
0,1.0,0.510605,0.362093,-0.003417,1.666299,0.362104
1,1.0,-0.119423,-0.318148,-151.289989,0.887432,22888.342554
2,1.0,1.227797,1.144017,-17.183399,3.413701,296.413213
3,1.0,0.439006,0.577195,-123.230617,1.551165,15186.362205
4,1.0,-0.008351,-0.287105,820.477948,0.991684,673183.776118


<a name="algorithms"></a>
<hr>
### Algorithm Setup

We begin with a quick function for catching singular matrix errors that we will use to decorate our Newton steps.

In [5]:
def catch_singularity(f):
    '''Silences LinAlg Errors and throws a warning instead.'''
    
    def silencer(*args, **kwargs):
        try:
            return f(*args, **kwargs)
        except np.linalg.LinAlgError:
            warnings.warn('Algorithm terminated - singular Hessian!')
            return args[0]
    return silencer

<a name="newton"></a>
<hr>
### Explanation of a Single Newton Step

Recall that Newton's method for maximizing / minimizing a given function $f(\beta)$ iteratively computes the following estimate:

$$
\beta^+ = \beta - Hf(\beta)^{-1}\nabla f(\beta)
$$

The Hessian of the log-likelihood for logistic regression is given by:
$$
Hf(\beta) = -X^TWX
$$
and the gradient is:
$$
\nabla f(\beta) = X^T(y-p)
$$
where
$$
W := \text{diag}\left(p(1-p)\right)
$$
and $p$ are the predicted probabilites computed at the current value of $\beta$.

### Connection to Iteratively Reweighted Least Squares (IRLS)
*For logistic regression, this step is actually equivalent to computing a weighted least squares estimator at each iteration!*  I.e.,
$$
\beta^+ = \arg\min_\beta (z-X\beta)^TW(z-X\beta)
$$
with $W$ as before and the *adjusted response* $z$ is given by
$$
z := X\beta + W^{-1}(y-p)
$$

**Takeaway:** This is fun, but in fact it can be leveraged to derive asymptotics and inferential statistics about the computed MLE $\beta^*$!

### Our implementations
Below we implement a single step of Newton's method, and we compute $Hf(\beta)^{-1}\nabla f(\beta)$ using `np.linalg.lstsq(A,b)` to solve the equation $Ax = b$.  Note that this does not require us to compute the actual inverse of the Hessian.

In [6]:
@catch_singularity
def newton_step(curr, X, lam=None):
    '''One naive step of Newton's Method'''
    
    ## compute necessary objects
    p = np.array(sigmoid(X.dot(curr[:,0])), ndmin=2).T
    W = np.diag((p*(1-p))[:,0])
    hessian = X.T.dot(W).dot(X)
    grad = X.T.dot(y-p)
    
    ## regularization
    if lam:
        step, *_ = np.linalg.lstsq(hessian + lam*np.eye(curr.shape[0]), grad)
    else:
        step, *_ = np.linalg.lstsq(hessian, grad)
        
    ## update
    beta = curr + step
    
    return beta

Next, we implement Newton's method in a *slightly* different way; this time, at each iteration, we actually compute the full inverse of the Hessian using `np.linalg.inv()`.

In [7]:
@catch_singularity
def alt_newton_step(curr, X, lam=None):
    '''One naive step of Newton's Method'''
    
    ## compute necessary objects
    p = np.array(sigmoid(X.dot(curr[:,0])), ndmin=2).T
    W = np.diag((p*(1-p))[:,0])
    hessian = X.T.dot(W).dot(X)
    grad = X.T.dot(y-p)
    
    ## regularization
    if lam:
        step = np.dot(np.linalg.inv(hessian + lam*np.eye(curr.shape[0])), grad)
    else:
        step = np.dot(np.linalg.inv(hessian), grad)
        
    ## update
    beta = curr + step
    
    return beta

<a name="conv"></a>
<hr>
### Convergence Setup

First we implement coefficient convergence; this stopping criterion results in convergence whenever
$$
\|\beta^+ - \beta\|_\infty < \epsilon
$$
where $\epsilon$ is a given tolerance.

In [8]:
def check_coefs_convergence(beta_old, beta_new, tol, iters):
    '''Checks whether the coefficients have converged in the l-infinity norm.
    Returns True if they have converged, False otherwise.'''
    
    coef_change = np.abs(beta_old - beta_new)
    
    return not (np.any(coef_change>tol) & (iters < max_iter))

Next we implement log-likelihood convergence.  This stopping criterion results in convergence whenever
$$
\left|\mathcal{L}(x,\beta^+) - \mathcal{L}(x,\beta)\right| < \epsilon
$$

In [9]:
def check_loglike_convergence(pred_probs, actuals, old_like, tol, iters):
    '''Checks whether the log-likelihood has converged.
    Returns True if converged, False otherwise.'''
    
    ## new log-likelihood value
    new_like = log_likelihood(pred_probs, actuals)
    
    return not ((np.abs(old_like-new_like)>tol) & (iters < max_iter))

<a name="numerics"></a>
<hr>
## Numerical Examples

### Standard Newton with Coefficient Convergence

In [10]:
## initial conditions
beta_old, beta = np.ones((len(X.columns),1)), np.zeros((len(X.columns),1))

iter_count = 0
coefs_converged = False

while not coefs_converged:
    
    beta_old = beta
    beta = newton_step(beta, X, lam=lam)
    iter_count += 1
    
    coefs_converged = check_coefs_convergence(beta_old, beta, tol, iter_count)
    
print('Iterations : {}'.format(iter_count))
print('Beta : {}'.format(beta))

Iterations : 11
Beta : [[  1.79082594e+17]
 [ -1.08285522e+18]
 [ -8.55920252e+17]
 [ -1.75727117e+18]
 [ -9.70100583e+17]
 [  5.18123533e+17]]


### Standard Newton with Log-likelihood Convergence

In [11]:
## initial conditions
beta = np.zeros((len(X.columns),1))
old_like, new_like = 1, 0
iter_count = 0
loglike_converged = False

while not loglike_converged:
    
    p = np.array(sigmoid(X.dot(beta[:,0])), ndmin=2).T
    old_like = log_likelihood(p, y)
    
    beta = newton_step(beta, X, lam=lam)
    p = np.array(sigmoid(X.dot(beta[:,0])), ndmin=2).T
    iter_count += 1
    
    loglike_converged = check_loglike_convergence(p, y, old_like, tol, iter_count)

    
print('Iterations : {}'.format(iter_count))
print('Beta : {}'.format(beta))

Iterations : 9
Beta : [[-1066.07747757]
 [-1398.19359417]
 [    4.88970121]
 [ -538.16955634]
 [ 2287.50721864]
 [  -87.95663379]]


### Alternate Newton with Log-likelihood Convergence

In [12]:
## initial conditions
beta = np.zeros((len(X.columns),1))
old_like, new_like = 1, 0
iter_count = 0

loglike_converged = False

while not loglike_converged:
    
    p = np.array(sigmoid(X.dot(beta[:,0])), ndmin=2).T
    old_like = log_likelihood(p, y)
    
    beta = alt_newton_step(beta, X, lam=lam)
    p = np.array(sigmoid(X.dot(beta[:,0])), ndmin=2).T
    iter_count += 1
    
    loglike_converged = check_loglike_convergence(p, y, old_like, tol, iter_count)
    
print('Iterations : {}'.format(iter_count))
print('Beta : {}'.format(beta))

Iterations : 9
Beta : [[-1066.07477481]
 [-1398.19101457]
 [    4.88957748]
 [ -538.16891391]
 [ 2287.50325827]
 [  -87.95644978]]


### Alternate Newton with Coefficient Convergence

In [13]:
## initial conditions
beta_old, beta = np.ones((len(X.columns),1)), np.zeros((len(X.columns),1))
iter_count = 0

coefs_converged = False

while not coefs_converged:
    
    beta_old = beta
    beta = alt_newton_step(beta, X, lam=lam)
    iter_count += 1
    
    coefs_converged = check_coefs_convergence(beta_old, beta, tol, iter_count)
    
print('Iterations : {}'.format(iter_count))
print('Beta : {}'.format(beta))

Iterations : 13
Beta : [[  2.92102353e+23]
 [ -7.43688983e+23]
 [  1.32775224e+24]
 [ -4.89685590e+22]
 [ -2.01242024e+23]
 [  2.77133562e+21]]


/Users/chriswhite1/anaconda3/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:8: UserWarning: Algorithm terminated - singular Hessian!


In [14]:
## if you want to check against statsmodels
# mod = smf.logit(formula, A).fit()
# mod.params